* Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

* Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

* Graficarlos.

* Obtener conclusiones.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = 'gdrive/My Drive/'
df = pd.read_csv('/content/drive/MyDrive/recetas.csv', on_bad_lines='skip', delimiter="|")

In [ ]:
df.head(10)

,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,NaN,baja,2.0,10m,NaN,https://www.recetasgratis.net/receta-de-balead...,0,0,2020-04-06,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,0,0,2020-03-27,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,0,0,2020-03-03,"3 tazas de harina de trigo (420 gramos),1 pizc..."
5,74068,Recetas de Aperitivos y tapas,Receta de Hummus de zanahoria,NaN,baja,6.0,1h 30m,Entrante,https://www.recetasgratis.net/receta-de-hummus...,0,0,2020-02-26,"2 zanahorias,1 chorro de aceite de olivia virg..."
6,74056,Recetas de Aperitivos y tapas,Receta de Tomates cherry confitados,NaN,baja,6.0,1h 30m,Acompañamiento,https://www.recetasgratis.net/receta-de-tomate...,0,0,2020-02-26,"500 gramos de tomates cherry,1 rama de romero ..."
7,74055,Recetas de Aperitivos y tapas,Receta de Ceviche de atún con mango,4.0,muy baja,4.0,1h 30m,Entrante,https://www.recetasgratis.net/receta-de-cevich...,1,4,2020-02-24,"2 filetes de atún,1 lima o limón,2 cucharadas ..."
8,74043,Recetas de Aperitivos y tapas,Receta de Edamame,NaN,muy baja,4.0,10m,Entrante,https://www.recetasgratis.net/receta-de-edamam...,0,0,2020-02-18,"1 bolsa de edamame (vainas de soja),1 pellizco..."
9,73727,Recetas de Aperitivos y tapas,Receta de Bechamel para croquetas,NaN,baja,3.0,15m,NaN,https://www.recetasgratis.net/receta-de-becham...,0,0,2020-02-04,"1 litro de leche,50 gramos de mantequilla,1 ce..."


In [ ]:
ingredientes = df.Ingredientes

In [ ]:
df = ingredientes.to_frame()

In [58]:
df = df.dropna()

In [59]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22150


Preprocessing

In [105]:
from keras.preprocessing.text import text_to_word_sequence

sentences = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentences.append(text_to_word_sequence(row[0]))

In [106]:
sentences[:1]

[['4',
  'trotillas',
  'de',
  'harina',
  '2',
  'tazas',
  'de',
  'frijoles',
  'negros',
  'cocidos',
  '2',
  'aguacates',
  '150',
  'gramos',
  'de',
  'queso',
  'rallado',
  '4',
  'cucharadas',
  'soperas',
  'de',
  'crema',
  'agria']]

In [122]:
#Quito numeros
sentence_tokens_no_num = []
for sentence in sentences:
  new_sentence = []
  for word in sentence:
    if not word.isnumeric():
       new_sentence.append(word)
  sentence_tokens_no_num.append(new_sentence)

In [155]:
#sentence_tokens_no_num[:1]

In [124]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [146]:
w2v_model = Word2Vec(min_count=5,    
                     window=2,       
                     size=50,       
                     negative=20,    
                     workers=1,      
                     sg=1)          

In [147]:
# Buildear vocabulario
w2v_model.build_vocab(sentence_tokens_no_num)

In [148]:
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 22150


In [149]:
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 2890


Training

In [150]:

w2v_model.train(sentence_tokens_no_num,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3354047.5
Loss after epoch 1: 2651292.5
Loss after epoch 2: 2546415.0
Loss after epoch 3: 2390246.0
Loss after epoch 4: 2377461.0
Loss after epoch 5: 2367420.0
Loss after epoch 6: 2287378.0
Loss after epoch 7: 2219630.0
Loss after epoch 8: 2214334.0
Loss after epoch 9: 2217522.0
Loss after epoch 10: 2209722.0
Loss after epoch 11: 2207938.0
Loss after epoch 12: 2205808.0
Loss after epoch 13: 2201440.0
Loss after epoch 14: 2142002.0
Loss after epoch 15: 2144168.0
Loss after epoch 16: 2138124.0
Loss after epoch 17: 2139488.0
Loss after epoch 18: 2142556.0
Loss after epoch 19: 2143436.0
Loss after epoch 20: 2144164.0
Loss after epoch 21: 2142064.0
Loss after epoch 22: 2147268.0
Loss after epoch 23: 2149796.0
Loss after epoch 24: 2141664.0
Loss after epoch 25: 2154988.0
Loss after epoch 26: 2152640.0
Loss after epoch 27: 2157880.0
Loss after epoch 28: 2160884.0
Loss after epoch 29: 1665536.0
Loss after epoch 30: 648448.0
Loss after epoch 31: 645576.0
Loss after epoch 32:

(19395335, 34353900)

Test

In [161]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["café"], topn=10)

[('instantáneo', 0.7104846835136414),
 ('soluble', 0.7014638781547546),
 ('cafe', 0.6901399493217468),
 ('nescafé', 0.6395068764686584),
 ('descafeinado', 0.6302001476287842),
 ('cacao', 0.6197483539581299),
 ('espresso', 0.6054309010505676),
 ('rasa', 0.5973159074783325),
 ('vainila', 0.5952202081680298),
 ('pistachos', 0.5874475240707397)]

Gráficos

In [156]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [157]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



**Conclusiones**

Elegí un dataset de recetas. Al no ser muy grande, tomé la decisión de entrenarlo con pocas dimensiones y un buen número de epochs. 

Observé que al traer las palabras más similares a palabras como papas, trae palabras que refieren a la familia de los tubérculos. 

Otro ejemplo interesante es el de "pasta" que en los primeros resultados trae distintos tipos de pasta: fusilli, espaghetti, ravioles. 

El dataset contiene muchos typos y los embeddings capturan la similaridad entre la palabra canónica y sus versiones mal escritas: 

Ejemplo: entre las palabras más similares a "pimienta" se encuentran "pimenta" y "pimineta". Es interesante ver que entre los más relacionados se encuentran otros condimentos. 
